In [1]:
from __future__ import print_function, division
import requests
import pandas as pd
import numpy as np
import os.path
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

%matplotlib inline

from sklearn.linear_model import LinearRegression, Lasso, LassoCV
import statsmodels.formula.api as smf

In [2]:
url2021 = 'https://www.basketball-reference.com/leagues/NBA_2021_per_game.html'
url2020 = 'https://www.basketball-reference.com/leagues/NBA_2020_per_game.html'
url2019 = 'https://www.basketball-reference.com/leagues/NBA_2019_per_game.html'
url2018 = 'https://www.basketball-reference.com/leagues/NBA_2018_per_game.html'
url2017 = 'https://www.basketball-reference.com/leagues/NBA_2017_per_game.html'
url2016 = 'https://www.basketball-reference.com/leagues/NBA_2016_per_game.html'
url2015 = 'https://www.basketball-reference.com/leagues/NBA_2015_per_game.html'
url2014 = 'https://www.basketball-reference.com/leagues/NBA_2014_per_game.html'
url2013 = 'https://www.basketball-reference.com/leagues/NBA_2013_per_game.html'
url2012 = 'https://www.basketball-reference.com/leagues/NBA_2012_per_game.html'

In [3]:
response = requests.get(url2021)
page = response.text
soup = BeautifulSoup(page, "lxml")

#Assigning header row to column
headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
headers = headers[1:]
#Web scrape text from table rows excluding header
rows = soup.find_all('tr')[1:]
player_stats = [[td.getText() for td in rows[i].find_all('td')]
            for i in range(len(rows))]
#Adding 2021 to df
statstotal = pd.DataFrame(player_stats, columns = headers)
statstotal['YEAR'] = 2021


In [4]:
url_list = [url2020, url2019, url2018, url2017, url2016, url2015, url2014, url2013, url2012]

In [5]:
for index, i in enumerate(url_list):
    response = requests.get(i)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    rows = soup.find_all('tr')[1:]
    player_stats2 = [[td.getText() for td in rows[r].find_all('td')] for r in range(len(rows))]
    stats = pd.DataFrame(player_stats2, columns = headers)
    stats['YEAR'] = 2021-(index+1)
    statstotal = statstotal.append(stats, ignore_index=True)

In [6]:
#Remove duplicate rows for players who were traded (keep only TOT)
statsclean = statstotal.drop_duplicates(subset = ['Player', 'YEAR'], keep='first')

In [7]:
import unicodedata

#Replace accented characters with normalized alphabet
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [8]:
statsclean['Player'] = statsclean['Player'].astype('string')

<ipython-input-8-6ba7d2e3ef18>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  statsclean['Player'] = statsclean['Player'].astype('string')


In [11]:
statsclean = statsclean.dropna()

In [13]:
statsclean['Player'] = statsclean['Player'].apply(remove_accents)

In [15]:
statsclean.to_csv('bballref.csv')